In [43]:
import os
import json
import pandas as pd
from PIL import Image
from tqdm import tqdm
import torch
from unsloth import FastLanguageModel
from transformers import AutoProcessor, TextStreamer
import nltk

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [44]:
import wandb

wandb.init(
    project="Prompting-Experiments",
    name="Prompt-Eval-Pixtral12B-CarDD",
    group="prompting-experiments",
    tags=["pixtral", "image-captioning", "prompting", "cardamage"],
    notes="Comparing handcrafted prompting strategies across multiple images and prompt types.",
    config={
        "model": "Pixtral-12B",
        "prompting": "handcrafted",
        "num_prompts": 6,
        "num_runs_per_prompt": 2,
        "dataset": "cardd",
    },
)

In [45]:
def create_image_caption_dataset(
    image_folder: str,
    captions_json: str,
    caption_strategy: str = 'first'
) -> pd.DataFrame:
    with open(captions_json, 'r') as f:
        captions_data = json.load(f)

    data = []
    for filename, caption_list in captions_data.items():
        image_path = os.path.join(image_folder, filename)
        if not os.path.exists(image_path):
            continue
        try:
            image = Image.open(image_path).convert("RGB")
            caption = caption_list if caption_strategy == 'first' else random.choice(caption_list)
            data.append({"image": image, "caption": caption, "filename": filename})
        except Exception as e:
            print(f"[ERROR] Could not load {filename}: {e}")
    return pd.DataFrame(data)

In [46]:
model_id = "unsloth/Pixtral-12B-2409"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

model = FastLanguageModel.for_inference(model)
processor = AutoProcessor.from_pretrained(model_id)

==((====))==  Unsloth 2025.5.3: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A40. Num GPUs = 1. Max memory: 44.448 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Mistral does not support SDPA - switching to eager!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [47]:
image_folder = "/workspace/data/test_dataset"
captions_json = "/workspace/data/test_set.json"

df = create_image_caption_dataset(image_folder, captions_json)
print(f"Loaded {len(df)} image-caption pairs")

Loaded 50 image-caption pairs


In [48]:
print("📌 First 5 entries:")
print(df.head(), "\n")

print("📋 Column types:")
print(df.dtypes)

print("\n🔎 Size")
print(df.shape)

📌 First 5 entries:
                                               image  \
0  <PIL.Image.Image image mode=RGB size=1000x667 ...   
1  <PIL.Image.Image image mode=RGB size=1000x667 ...   
2  <PIL.Image.Image image mode=RGB size=1000x667 ...   
3  <PIL.Image.Image image mode=RGB size=1000x750 ...   
4  <PIL.Image.Image image mode=RGB size=1000x667 ...   

                                             caption    filename  
0  The car exhibits visible damage including a sc...  000481.jpg  
1  The car exhibits significant damage to the win...  000433.jpg  
2  The car exhibits a flat tire on the rear wheel...  000749.jpg  
3  The car exhibits significant damage. The rear ...  000541.jpg  
4  The car shows scratches along the side panel a...  000424.jpg   

📋 Column types:
image       object
caption     object
filename    object
dtype: object

🔎 Size
(50, 3)


In [49]:
import torch
import time
from transformers import TextStreamer
import pandas as pd

streamer = TextStreamer(tokenizer, skip_prompt=True)

def run_vlm_inference(prompt: str, filename: str, df: pd.DataFrame):
    """
    Perform inference on a given image (by filename) from the dataframe using a custom prompt.

    Args:
        prompt (str): The prompt text (can include mask tokens)
        filename (str): Filename of the image in the DataFrame
        df (pd.DataFrame): DataFrame with 'filename' and 'image' columns

    Returns:
        Tuple[str, float, float]: (Generated output, inference time in seconds, VRAM used in GB)
    """
    row = df[df["filename"] == filename]
    if row.empty:
        print(f"[ERROR] No image found with filename: {filename}")
        return None, 0.0, 0.0

    row = row.iloc[0]
    image = row["image"]
    caption = row["caption"]

    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {"type": "image", "image": image}
            ]
        }
    ]

    input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor(images=image, text=input_text, return_tensors="pt").to("cuda")

    torch.cuda.reset_peak_memory_stats()
    start_mem = torch.cuda.memory_allocated() / 1024 / 1024 / 1024  # in GB
    start_time = time.time()

    print(f"🔹 Image: {filename}")
    print(f"🧾 Prompt: {prompt}")
    print("📤 Output:")
    
    # Perform inference
    output_ids = model.generate(
        **inputs,
        streamer=streamer,
        max_new_tokens=128,
        use_cache=True,
        temperature=1.5,
        min_p=0.1,
    )

    end_time = time.time()
    end_mem = torch.cuda.max_memory_allocated() / 1024**3

    time_taken = round(end_time - start_time, 3)
    vram_used = round(end_mem - start_mem, 3)
    decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    print(f"⏱️ Time taken: {time_taken} sec | 🧠 VRAM used: {vram_used} GB")
    print("-" * 80)

    return decoded_output, time_taken, vram_used

In [51]:
# List of filenames for car damage dataset
test_filenames = [
    "000404.jpg", "000422.jpg", "000433.jpg", "000481.jpg", "000520.jpg",
    "000541.jpg", "000698.jpg", "000740.jpg", "000869.jpg", "000889.jpg"
]

# Prompt set used for all filenames
CAR_DAMAGE_PROMPTS = [
    "",  # No Prompt
    "Describe &&damage 12 sedan drive’ this !!image.",  # Noisy
    "An image of a damaged car parked on the side of the road.",  # Hand-crafted
    "You are an insurance claims assessor. Provide a detailed description of the car’s condition.",  # Roleplay
    "This <part_1> of the car has <damage_type_1>. The severity appears to be <severity_1>. Additional notes: <text_1>.",  # Masked
    "Describe using format - Damage Type: ___; Affected Part: ___; Severity: ___; Notes: ___"  # Format-Guided
]

all_outputs = {}
all_times = {}
all_vram = {}

for filename in test_filenames:
    row = df[df["filename"] == filename]
    if row.empty:
        print(f"[WARNING] No data found for {filename}")
        continue

    print(f"\n🔍 Running prompts for image: {filename}\n" + "-"*60)

    inference_outputs = []
    inference_times = []
    vram_usages = []

    for prompt in CAR_DAMAGE_PROMPTS:
        for run in range(2):  # Run each prompt twice
            print(f"🔁 Run {run+1} for prompt: {prompt[:50]}{'...' if len(prompt) > 50 else ''}")
            output, time_taken, vram_used = run_vlm_inference(prompt, filename, df=df)
            inference_outputs.append(output)
            inference_times.append(time_taken)
            vram_usages.append(vram_used)

    # Store in dicts
    all_outputs[filename] = inference_outputs
    all_times[filename] = inference_times
    all_vram[filename] = vram_usages



🔍 Running prompts for image: 000404.jpg
------------------------------------------------------------
🔁 Run 1 for prompt: 
🔹 Image: 000404.jpg
🧾 Prompt: 
📤 Output:
The hood of the hood ornamelizabethada{,} 1

 ekst-–––whatever your browser or the chooser: the in unreal-est and `1lea ithisize myf5. of tI [on;).
ose inth of theinns. Def    I- the the the the the the a ized14.</s>
⏱️ Time taken: 5.507 sec | 🧠 VRAM used: 0.601 GB
--------------------------------------------------------------------------------
🔁 Run 2 for prompt: 
🔹 Image: 000404.jpg
🧾 Prompt: 
📤 Output:
The hood of the hood ornamelizabethada{,} 1

 ekst-–––whatever your browser or the chooser: the in unreal-est and `1lea ithisize myf5. of tI [on;).
ose inth of theinns. Def    I- the the the the the the a ized14.</s>
⏱️ Time taken: 5.886 sec | 🧠 VRAM used: 0.601 GB
--------------------------------------------------------------------------------
🔁 Run 1 for prompt: Describe &&damage 12 sedan drive’ this !!image.
🔹 Image: 000

In [52]:
from sentence_transformers import SentenceTransformer, util
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.spice.spice import Spice
from pycocoevalcap.tokenizer.ptbtokenizer import PTBTokenizer
import pandas as pd

sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

In [53]:
def compute_cosine_similarity(reference_captions, generated_caption):
    try:
        total_score = 0.0
        for caption in reference_captions:
            ref_embed = sbert_model.encode(caption, convert_to_tensor=True)
            gen_embed = sbert_model.encode(generated_caption, convert_to_tensor=True)
            score = util.cos_sim(gen_embed, ref_embed).item()
            total_score += score
        avg_score = total_score / len(reference_captions) if reference_captions else 0.0
        return avg_score
    except Exception as e:
        print(f"Error computing cosine similarity: {e}")
        return 0.0

In [54]:
def compute_cider_spice_scores(reference_caption, generated_caption):
    refs = {0: [reference_caption if reference_caption else ""]}
    hypos = {0: [generated_caption if generated_caption else ""]}

    # print(f"Generated caption: {generated_caption}")
    # print(f"Generated hypos: {hypos}")

    ptb = PTBTokenizer()
    refs_tok = ptb.tokenize({i: [{"caption": c} for c in caps] for i, caps in refs.items()})
    hypos_tok = ptb.tokenize({i: [{"caption": hypos[i][0]}] for i in hypos})

    all_scores = {}

    for scorer, name in [(Cider(), "CIDEr"), (Spice(), "SPICE")]:
        try:
            avg_score, _ = scorer.compute_score(refs_tok, hypos_tok)
            if name == "SPICE":
                # SPICE returns dicts per image
                all_scores[name] = avg_score.get("All", {}).get("f", 0.0) if isinstance(avg_score, dict) else avg_score
            else:
                all_scores[name] = avg_score
        except Exception as e:
            print(f"[ERROR] {name} scoring failed: {e}")
            all_scores[name] = 0.0

    return all_scores

In [55]:
from cider import Cider

PICKLE_PATH = "/workspace/data/cardd-df.p"

def compute_cider2_score(reference_caption, generated_caption):
    """
    Computes CIDEr score using new evaluate_cider logic with precomputed DF pickle.

    Args:
        reference_caption (str or list): Ground truth caption(s)
        generated_caption (str): Model output

    Returns:
        float: CIDEr score (averaged if multiple refs)
    """
    refs = {"0": reference_caption if isinstance(reference_caption, list) else [reference_caption]}
    hypos = [{"image_id": "0", "caption": [generated_caption]}]  # Fix: caption should be a list

    cider = Cider()
    score, _ = cider.compute_score(refs, hypos, PICKLE_PATH)

    return score

In [56]:
def evaluate_all_metrics(reference_caption, generated_caption):
    cider_spice_scores = compute_cider_spice_scores(reference_caption, generated_caption)
    cosine_sim = compute_cosine_similarity([reference_caption], generated_caption)
    cider2 = compute_cider2_score(reference_caption, generated_caption)

    return {
        "cosine_similarity": round(cosine_sim, 4),
        #"CIDEr": round(cider_spice_scores.get("CIDEr", 0.0), 4),
        "SPICE": round(cider_spice_scores.get("SPICE", 0.0), 4),
        "CIDEr": round(cider2, 4)
    }

In [57]:
all_metrics_scores = {}

for filename, outputs in all_outputs.items():  # filename is used instead of img_id
    row = df[df["filename"] == filename]
    if row.empty:
        print(f"[WARNING] No ground truth found for filename {filename}")
        continue

    ground_truth = row.iloc[0]["caption"]  # Single caption only
    print(f"\n🔍 Evaluating image: {filename} | Reference Caption:\n{ground_truth}\n{'-'*80}")
    
    scores_for_image = []

    for idx, gen_output in enumerate(outputs):  # Expecting 2 runs per image
        metrics = evaluate_all_metrics(ground_truth, gen_output)
        avg_metrics = {
            "CIDEr": round(metrics["CIDEr"], 4),
            "SPICE": round(metrics["SPICE"], 4),
            "cosine_similarity": round(metrics["cosine_similarity"], 4)
        }

        print(f"📝 Prompt Run {idx+1} | CIDEr: {avg_metrics['CIDEr']} | SPICE: {avg_metrics['SPICE']} | CosSim: {avg_metrics['cosine_similarity']}")
        scores_for_image.append(avg_metrics)

    all_metrics_scores[filename] = scores_for_image



🔍 Evaluating image: 000404.jpg | Reference Caption:
The car exhibits significant damage. The front hood area shows a large dent with visible deformation and creasing. The headlight on the passenger's side is broken, with the lamp housing shattered and the bulb exposed. The front bumper also has a dent near the grille, and the grille itself appears to be slightly misaligned. The passenger side headlight is intact but the surrounding area has a dent. The tire on the driver's side appears to be flat. No scratches, cracks, or shattered glass are present. The overall condition of the car is severely compromised due to these damages.
--------------------------------------------------------------------------------


PTBTokenizer tokenized 112 tokens at 1308.55 tokens per second.
PTBTokenizer tokenized 54 tokens at 1248.22 tokens per second.


[ERROR] CIDEr scoring failed: Cider.compute_score() missing 1 required positional argument: 'pfile_path'


Parsing reference captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [0.5 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [1.2 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [0.7 sec].
Loading classif

SPICE evaluation took: 13.26 s
📝 Prompt Run 1 | CIDEr: 5.3422 | SPICE: 0.0 | CosSim: 0.185


...

SPICE evaluation took: 12.17 s
📝 Prompt Run 11 | CIDEr: 0.132 | SPICE: 0.0 | CosSim: 0.4119


PTBTokenizer tokenized 34 tokens at 829.78 tokens per second.
PTBTokenizer tokenized 136 tokens at 3216.18 tokens per second.


[ERROR] CIDEr scoring failed: Cider.compute_score() missing 1 required positional argument: 'pfile_path'


Parsing reference captions
Parsing test captions


SPICE evaluation took: 706.8 ms
📝 Prompt Run 12 | CIDEr: 0.132 | SPICE: 0.0 | CosSim: 0.4119


In [58]:
import pandas as pd

for img_id, scores in all_metrics_scores.items():
    print(f"\n📸 Image ID: {img_id}")
    print("-" * 60)

    df_temp = pd.DataFrame(scores)
    prompt_labels = []
    for i in range(len(scores)):
        prompt_num = (i // 2) + 1  # Prompts 1–6
        run_num = (i % 2) + 1      # Run 1 or Run 2
        prompt_labels.append(f"Prompt {prompt_num} (Run {run_num})")

    df_temp.index = prompt_labels
    print(df_temp.round(4))
    print("=" * 60)


📸 Image ID: 000404.jpg
------------------------------------------------------------
                   CIDEr   SPICE  cosine_similarity
Prompt 1 (Run 1)  5.3422  0.0000             0.1850
Prompt 1 (Run 2)  5.3422  0.0000             0.1850
Prompt 2 (Run 1)  0.5702  0.0317             0.4061
Prompt 2 (Run 2)  0.5702  0.0317             0.4061
Prompt 3 (Run 1)  1.1607  0.0455             0.3116
Prompt 3 (Run 2)  1.1607  0.0455             0.3116
Prompt 4 (Run 1)  3.3462  0.0274             0.2931
Prompt 4 (Run 2)  3.3462  0.0274             0.2931
Prompt 5 (Run 1)  5.8958  0.0286             0.5898
Prompt 5 (Run 2)  5.8958  0.0286             0.5898
Prompt 6 (Run 1)  1.1225  0.0000             0.5004
Prompt 6 (Run 2)  1.1225  0.0000             0.5004

📸 Image ID: 000422.jpg
------------------------------------------------------------
                   CIDEr   SPICE  cosine_similarity
Prompt 1 (Run 1)  2.9109  0.0000             0.1561
Prompt 1 (Run 2)  2.9109  0.0000             0.156

In [59]:
# Log results table
table = wandb.Table(columns=["filename", "prompt_type", "run", "image", "ground_truth", "prediction"])

for filename in all_outputs:
    row = df[df["filename"] == filename]
    if row.empty:
        continue

    ground_truth = row.iloc[0]["caption"]
    image_pil = row.iloc[0]["image"]

    for prompt_index in range(6):
        for run in range(2):
            row_idx = prompt_index * 2 + run
            prediction = all_outputs[filename][row_idx] if row_idx < len(all_outputs[filename]) else "N/A"

            table.add_data(
                filename,
                f"Prompt {prompt_index + 1}",
                f"Run {run + 1}",
                wandb.Image(image_pil),
                ground_truth,
                prediction
            )

wandb.log({"Prompting Comparison Results": table})

# Log metrics for each prompt run
for filename, scores in all_metrics_scores.items():
    for i, score in enumerate(scores):
        wandb.log({
            f"{filename}/Prompt {i//2 + 1}/Run {i%2 + 1}/CIDEr": score["CIDEr"],
            f"{filename}/Prompt {i//2 + 1}/Run {i%2 + 1}/SPICE": score["SPICE"],
            f"{filename}/Prompt {i//2 + 1}/Run {i%2 + 1}/Cosine": score["cosine_similarity"]
        })


In [60]:
import pandas as pd
from openpyxl import Workbook, load_workbook
from openpyxl.drawing.image import Image as ExcelImage
from openpyxl.styles import Alignment
from PIL import Image as PILImage
import os

def log_prompt_metrics_to_excel(
    filename: str,
    model_name: str,
    inference_outputs: list,
    metrics: list,
    inference_times: list,
    vram_usages: list,
    df: pd.DataFrame,
    output_excel_path: str = "prompt_tuning_results_cardd_pixtral.xlsx"
):
    row = df[df["filename"] == filename].iloc[0]
    original_caption = row["caption"]
    image_path = os.path.join("/workspace/data/test_dataset", filename)

    # Load or create workbook
    if os.path.exists(output_excel_path):
        wb = load_workbook(output_excel_path)
        ws = wb.active
    else:
        wb = Workbook()
        ws = wb.active
        ws.title = "Prompt Evaluation"
        headers = [
            "Model", "Image Number", "Image", "Original Caption",
            "Prompt", "Output", "Inference Time (s)", "VRAM Used (GB)",
            "CIDEr", "SPICE", "Cosine Similarity"
        ]
        ws.append(headers)

    # Loop over the 2 runs (Run 1, Run 2) — 6 prompts per run
    for run_index in range(2):  # 0 for Run 1, 1 for Run 2
        start_row = ws.max_row + 1

        for prompt_index in range(6):  # Prompt 1 to 6
            idx = prompt_index * 2 + run_index
            ws.append([
                model_name,
                filename,
                "",  # Placeholder for image
                original_caption,
                f"Prompt {prompt_index + 1} (Run {run_index + 1})",
                inference_outputs[idx],
                inference_times[idx],
                vram_usages[idx],
                metrics[idx]["CIDEr"],
                metrics[idx]["SPICE"],
                metrics[idx]["cosine_similarity"]
            ])

        # Merge A–D columns across 6 rows
        for col in ["A", "B", "C", "D"]:
            ws.merge_cells(f"{col}{start_row}:{col}{start_row + 5}")

        # Apply alignment and formatting
        align_top_wrap = Alignment(wrap_text=True, vertical="top")
        align_top = Alignment(vertical="top")

        for row_idx in range(start_row, start_row + 6):
            for col_letter in ["A", "B", "C", "E"]:
                ws[f"{col_letter}{row_idx}"].alignment = align_top
            for col_letter in ["D", "F"]:
                ws[f"{col_letter}{row_idx}"].alignment = align_top_wrap
            ws.row_dimensions[row_idx].height = 120

        # Set column widths
        ws.column_dimensions["D"].width = 40  # Original Caption
        ws.column_dimensions["F"].width = 40  # Output

        # Embed image in this run block
        if os.path.exists(image_path):
            print(f"[INFO] Inserting image for {filename} | Run {run_index + 1}")
            try:
                img = ExcelImage(image_path)
                img.width = 150
                img.height = 150
                img.anchor = f"C{start_row}"
                ws.add_image(img)
            except Exception as e:
                print(f"[ERROR] Could not insert image: {e}")

    # Save workbook
    wb.save(output_excel_path)
    print(f"✅ Logged both runs for {filename} to {output_excel_path}")

In [61]:
for filename in all_outputs.keys():
    log_prompt_metrics_to_excel(
        filename=filename,
        model_name=model_id,
        inference_outputs=all_outputs[filename],
        metrics=all_metrics_scores[filename],
        inference_times=all_times[filename],
        vram_usages=all_vram[filename],
        df=df
    )

[INFO] Inserting image for 000404.jpg | Run 1
[INFO] Inserting image for 000404.jpg | Run 2
✅ Logged both runs for 000404.jpg to prompt_tuning_results_cardd_pixtral.xlsx
[INFO] Inserting image for 000422.jpg | Run 1
[INFO] Inserting image for 000422.jpg | Run 2
✅ Logged both runs for 000422.jpg to prompt_tuning_results_cardd_pixtral.xlsx
[INFO] Inserting image for 000433.jpg | Run 1
[INFO] Inserting image for 000433.jpg | Run 2
✅ Logged both runs for 000433.jpg to prompt_tuning_results_cardd_pixtral.xlsx
[INFO] Inserting image for 000481.jpg | Run 1
[INFO] Inserting image for 000481.jpg | Run 2
✅ Logged both runs for 000481.jpg to prompt_tuning_results_cardd_pixtral.xlsx
[INFO] Inserting image for 000520.jpg | Run 1
[INFO] Inserting image for 000520.jpg | Run 2
✅ Logged both runs for 000520.jpg to prompt_tuning_results_cardd_pixtral.xlsx
[INFO] Inserting image for 000541.jpg | Run 1
[INFO] Inserting image for 000541.jpg | Run 2
✅ Logged both runs for 000541.jpg to prompt_tuning_results

In [62]:
wandb.finish()

000404.jpg/Prompt 1/Run 1/CIDEr,▁
000404.jpg/Prompt 1/Run 1/Cosine,▁
000404.jpg/Prompt 1/Run 1/SPICE,▁
000404.jpg/Prompt 1/Run 2/CIDEr,▁
000404.jpg/Prompt 1/Run 2/Cosine,▁
000404.jpg/Prompt 1/Run 2/SPICE,▁
000404.jpg/Prompt 2/Run 1/CIDEr,▁
000404.jpg/Prompt 2/Run 1/Cosine,▁
000404.jpg/Prompt 2/Run 1/SPICE,▁
000404.jpg/Prompt 2/Run 2/CIDEr,▁
000404.jpg/Prompt 2/Run 2/Cosine,▁
